### Simple Solution

In [1]:
import pandas as pd

#### 1. Forecast: We start with a forecast
For each store and item:

Calculation:
* Forecast - stockfile  = amount_to_purchase

Problem:
* The stockfile is unreliable. **Note:** Look at where stockfile is recorded.
* Suppliers under and over-deliver

Limitation:
* amount_to_purchase can only be in tray sizes.
* Must order enough to cover delivery period as suppliers can't deliver every day. **Note:** Check the range of delivery periods.

#### 2. Depot: Supplier delivers to the depots

Problem:
* There will be a difference between the shipped amount from a depot and the amount arriving at a store

Limitation:
* The allocation to a shop is limited by the supply to the depot.

#### 3. In Store: Products arrive at the store

Problem:
* Delivered amount can differ from depot shipping amount - incorrectly picked

#### 4. Closing Inventory:

Closing inventory is also taken in store and all its fields are contained in the In Store data.

**Note:** Check if they are the same. If same, discard, closing inventory. If not, give measure of difference and check with company why there are two and which is most reliable.

In [2]:
fore = pd.read_csv('Downloads/Case Study - Forecast Data.txt', sep='\t', encoding='utf-16',
                  parse_dates=['calendar_date'])

In [3]:
depot = pd.read_csv('Downloads/Case Study - Depot Data.txt', sep='\t', encoding='utf-16',
                   parse_dates=['calendar_date'])

In [4]:
instore = pd.read_csv('Downloads/Case Study - In Store Data.txt', sep='\t', encoding='utf-16',
                     parse_dates=['calendar_date'])

In [5]:
cinv = pd.read_csv('Downloads/Case Study - Closing Inventory.txt', sep='\t', encoding='utf-16',
                   parse_dates=['calendar_date'])

### Data checks

Check keys: the key is made of `upc`, `calendar_date`, `store_id`, `geography_id`

* Which products are in all files? Only include products which we have complete data set for
* Do store IDs appear in mulitple geographys?
* Is `calendar_date` consistent across all files. Only include date range where we have all data.

In [6]:
fore.shape[0] == depot.shape[0] == cinv.shape[0] == instore.shape[0]

True

Same number of rows in all slices

#### Date range

In [7]:
def return_daterange(df):
    return df['calendar_date'].min(), df['calendar_date'].max()

In [8]:
return_daterange(fore)

(Timestamp('2018-01-08 00:00:00'), Timestamp('2018-12-07 00:00:00'))

In [9]:
return_daterange(instore)

(Timestamp('2018-01-08 00:00:00'), Timestamp('2018-12-07 00:00:00'))

#### Stores in multiple geographies?

In [10]:
(fore.groupby('store_id')['geography_id'].nunique() == 1).value_counts()

True    623
Name: geography_id, dtype: int64

In [11]:
(cinv.groupby('store_id')['geography_id'].nunique() == 1).value_counts()

True    623
Name: geography_id, dtype: int64

In [12]:
(depot.groupby('store_id')['geography_id'].nunique() == 1).value_counts()

True    623
Name: geography_id, dtype: int64

In [13]:
pd.merge(pd.DataFrame(fore['upc'].drop_duplicates()), pd.DataFrame(depot['upc'].drop_duplicates()), how='outer').shape

(143, 1)

All files have:
* dates between 08.01.2018 and 07.12.2018
* Store IDs that only one appear in one geography
* 143 products

#### Is Closing Inv the same In Store?

In [14]:
(cinv == instore[cinv.columns]).apply(lambda x: x.value_counts()).transpose()

,False,True
upc,NaN,2648539.0
calendar_date,NaN,2648539.0
calendar_id,NaN,2648539.0
store_id,NaN,2648539.0
geography_id,NaN,2648539.0
shelf_life,NaN,2648539.0
units_per_tray,NaN,2648539.0
closing_inventory_min_neg_over_shelf_life_minus_2_days,2554345.0,94194.0
closing_inventory_neg_count_over_1_day,NaN,2648539.0
closing_inventory_neg_count_over_shelf_life_minus_2_days,NaN,2648539.0


**Note:** Closing Inventory and In Store are identical except for:
* closing_inventory_min_neg_over_shelf_life_minus_2_days
* closing_inventory_on_day

Join these to final data with `cinv` prefix

In [15]:
cinv = cinv.rename(columns={'closing_inventory_min_neg_over_shelf_life_minus_2_days' :
                           'cinv_closing_inventory_min_neg_over_shelf_life_minus_2_days',
                           'closing_inventory_on_day' : 
                           'cinv_closing_inventory_on_day'})

In [16]:
cinv_cols = ['upc', 'calendar_date', 'store_id', 'geography_id', 'cinv_closing_inventory_min_neg_over_shelf_life_minus_2_days',
            'cinv_closing_inventory_on_day']
join_cols = ['upc', 'calendar_date', 'store_id', 'geography_id']
instore_c = instore.merge(cinv[cinv_cols], on=join_cols).drop(['calendar_id'], axis=1)

In [17]:
# Are the consistent columns the same in depot and instore
mut_cols = list(depot.columns[depot.columns.isin(instore_c.columns)])
(depot[mut_cols] == instore[mut_cols]).apply(lambda x: x.value_counts()).transpose()
# or all(depot[mut_cols] == instore[mut_cols])
# yes

,True
upc,2648539
calendar_date,2648539
store_id,2648539
geography_id,2648539
shelf_life,2648539
units_per_tray,2648539


In [19]:
depot_cols = ['upc', 'calendar_date', 'store_id', 'geography_id', 'depot_delivered_qty_on_day',
       'depot_delivered_qty_over_minus_2_day',
       'depot_delivered_qty_over_shelf_life_plus_1',
       'depot_lvl_required_qty_over_supplier_lead_time',
       'depot_lvl_target_inventory_on_day', 'depot_ordered_qty_on_day',
       'depot_ordered_qty_over_minus_2_day',
       'depot_ordered_qty_over_shelf_life_plus_1',
       'depot_ordered_qty_over_supplier_lead_time', 'depot_store_id']

In [20]:
instorec_dep = instore_c.merge(depot[depot_cols], on=['upc', 'calendar_date', 'store_id', 'geography_id'])

#### Check same columns forecast

In [22]:
# Are the consistent columns the same in depot and instore
mut_cols = list(fore.columns[fore.columns.isin(instorec_dep.columns)])
#(fore[mut_cols] == instorec_dep[mut_cols]).apply(lambda x: x.value_counts()).transpose()
all(depot[mut_cols] == instore[mut_cols])
# yes - no oddities around shelf life and tray size

True

In [23]:
joined_data = instorec_dep.merge(fore.drop(columns=['calendar_id', 'shelf_life', 'units_per_tray'], axis=1),
                    on=['upc', 'calendar_date', 'store_id', 'geography_id'])

### Assert

In [24]:
assert joined_data.shape[0] == fore.shape[0] == depot.shape[0] == cinv.shape[0] == instore.shape[0]

**Stop:** Write quick data Dictionary and review values

In [25]:
joined_data.to_csv('/Users/chughes/Documents/Projects/Newton Europe/joined_data.csv', index=False)

In [26]:
pd.DataFrame(joined_data.columns.transpose()).to_excel(
    '/Users/chughes/Documents/Projects/Newton Europe/data_dict.xlsx', index=False)

### Logic

In [27]:
# Waste qty > 0 on days when ranging indicator is 0

In [44]:
# Waste qty > 0 on days when ranging indicator is 0
joined_data['product_waste'] = (joined_data['waste_value_on_day'] > 0) & (joined_data['ranging_indicator_on_day'] == 0)

In [48]:
# Stockout indicator 1 on a day when ranging indicator is 0
joined_data['product_stockout'] = (joined_data['stock_out_ind_on_day'] == 1) & (joined_data['ranging_indicator_on_day'] == 0)

In [55]:
# Received_units (store) > Allocation (store) Shelf life before and including waste event
joined_data['delivered_qty_over_shelf_life'] > joined_data['allocated_qty_over_shelf_life']

0           True
1          False
2          False
3          False
4          False
           ...  
2648534    False
2648535    False
2648536    False
2648537    False
2648538    False
Length: 2648539, dtype: bool

In [61]:
# Received_units (store) < Allocation (store) Allocation to store lead time before and including stockout
(joined_data['delivered_qty_over_shelf_life'] < joined_data['allocated_required_diff_high_qty_over_lead_time'])

0          False
1          False
2          False
3          False
4          False
           ...  
2648534    False
2648535    False
2648536    False
2648537    False
2648538    False
Length: 2648539, dtype: bool

In [79]:
### Sales
# Sales Forecast Error
joined_data['forecast_waste'] = (((joined_data['forecast_demand_qty_over_shelf_life'] - joined_data['sales_qty_over_shelf_life'])) > (
    0.75*joined_data['forecast_demand_qty_over_shelf_life'])) & (
    (joined_data['forecast_demand_qty_over_shelf_life'] - joined_data['sales_qty_over_shelf_life']).abs() > 1
)


# Forecast Stockout
joined_data['forecast_stockout'] = ((joined_data['forecast_demand_qty_over_lead_time'] - joined_data['sales_qty_over_lead_time']) < (
    1.25*joined_data['forecast_demand_qty_over_lead_time'])) & (
    (joined_data['forecast_demand_qty_over_lead_time']- joined_data['sales_qty_over_lead_time']).abs() > 1
)

In [82]:
### Waste
joined_data['waste_waste'] = ((joined_data['forecast_waste_qty_over_shelf_life'] - joined_data['waste_value_on_day']) > (
    0.75*joined_data['forecast_waste_qty_over_shelf_life']))

In [83]:
joined_data['waste_stockout'] = (
    joined_data['forecast_waste_qty_over_lead_time'] < (joined_data['waste_value_on_day'] - 0.75*joined_data['forecast_waste_qty_over_lead_time']) 
)

In [84]:
### Stock
# Stock file adjustment
joined_data['stock_waste'] = (joined_data['stockfile_adjust_qty_on_day'] > 0) & (joined_data['stockfile_adjust_qty_at_plus_1_day'] > 0)

# Stock had less stock than reported
joined_data['stock_stockout'] = (joined_data['stockfile_adjust_qty_on_day'] < 0) & (
joined_data['stockfile_adjust_qty_at_minus_1_day'] < 0)

In [ ]:
### Negative stockfile
    
(joined_data['closing_inventory_on_day'] < 0) & (joined_data['clos'] < 0)

In [90]:
joined_data['closing_inventory_min_neg_over_shelf_life_minus_2_days'].isna().value_counts()

True     2554345
False      94194
Name: closing_inventory_min_neg_over_shelf_life_minus_2_days, dtype: int64

In [93]:
joined_data['closing_inventory_neg_count_over_shelf_life_minus_2_days'].value_counts()

0     2554345
1       72726
2       14511
3        3832
4        1395
5         485
8         393
6         377
7         210
9         179
13         33
10         24
11         16
12         13
Name: closing_inventory_neg_count_over_shelf_life_minus_2_days, dtype: int64

In [88]:
joined_data[['closing_inventory_min_neg_over_shelf_life_minus_2_days','closing_inventory_neg_count_over_shelf_life_minus_2_days']]

,closing_inventory_min_neg_over_shelf_life_minus_2_days,closing_inventory_neg_count_over_shelf_life_minus_2_days
0,NaN,0
1,NaN,0
2,NaN,0
3,NaN,0
4,NaN,0
...,...,...
2648534,NaN,0
2648535,NaN,0
2648536,NaN,0
2648537,NaN,0
